In [3]:
# Import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score,precision_score,recall_score
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [7]:
# Import datasets
df_train = pd.read_csv('assets/df_train.csv',index_col=0)
df_test = pd.read_csv('assets/df_test.csv',index_col=0)

In [19]:
vec_tfidf = TfidfVectorizer(ngram_range=(1, 1), max_df=0.8, min_df=3,max_features=5000)
X_train_tfidf = vec_tfidf.fit_transform(df_train.tweet_text)

In [26]:
unique_labels = df_train['class_label'].unique()
dict_label = dict(zip(unique_labels,np.arange(len(unique_labels))))
df_train['class_label_ordinal'] = df_train['class_label'].map(dict_label)

In [27]:
df_train

,tweet_id,tweet_text,class_label,place,disaster,year,class_label_ordinal
0,735891446960623616,RT @DonBradshawNTV: How @MarshallAmpsUK came t...,other_relevant_information,canada,wildfires,2016,0
1,731202020296818688,Red Cross distributes $30M to Fort McMurray wi...,displaced_people_and_evacuations,canada,wildfires,2016,1
2,733665357236342784,Interesting insights on the shifting communica...,other_relevant_information,canada,wildfires,2016,0
3,731963038429929472,RT @globeandmail: Oil sands producers helping ...,rescue_volunteering_or_donation_effort,canada,wildfires,2016,2
4,728674838034944001,Ottawa to match Red Cross donations for Fort M...,rescue_volunteering_or_donation_effort,canada,wildfires,2016,2
...,...,...,...,...,...,...,...
1365,1176581379657785345,26 confirmed dead &amp; scores injured as 5.8 ...,injured_or_dead_people,pakistan,earthquake,2019,8
1366,1176507658590916608,#Breaking Injured are being shifted to hospita...,injured_or_dead_people,pakistan,earthquake,2019,8
1367,1176719506137370624,Always that one sher jawan who doesn’t leave t...,other_relevant_information,pakistan,earthquake,2019,0
1368,1176496209185431553,Reports of deaths and dozens of injuries comin...,injured_or_dead_people,pakistan,earthquake,2019,8
